In [1]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
import matplotlib.pyplot as plt

* 완전계층 (Affine 계층)은 2차원 이상의 데이터를 flatten하기 때문에 데이터의 공간적 정보가 무시됨
* CNN은 형상을 유지하기 때문에 이러한 정보를 살릴 수 있음

## Ⅰ. 합성곱 연산

### 1. Kernel (Filter)

* Fully connected layer에서의 가중치 역할과 유사하다

<img src='../img/fig 7-3.png' width=50%, height=50%/>

<img src='../img/fig 7-4.png' width=50%, height=50%/>

### 2. Bias

* Fully connected layer에서의 bias 역할과 유사하다
* bias를 부여하는 방식은 2가지가 있음
  * Tied bias: 각 출력 특성맵 당 하나의 bias, 즉 한 특성맵의 모든 픽셀에서 bias는 동일하다 <= 일반적으로 이 방식을 쓴다
  * Untied bias: 각 출력 특성맵의 픽셀 당 하나의 bias

<img src='../img/fig 7-5.png' width=70%, height=70%/>

### 3. Padding

* 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로(보통 0)으로 채우는 것
* 쓰는 이유
  * 출력 크기 조정
  * 테두리 정보 보존

<img src='../img/fig 7-6.png' width=50%, height=50%/>

### 3. Stride

* 필터를 적용하는 위치의 간격

<img src='../img/fig 7-7.png' width=50% height=50%/>

### 4. 관계식

$$
\text{input dimension} = (h_i, w_i) \quad \text{kernel size} = (h_k, w_k) \quad \text{padding} = (p_h, p_w) \quad \text{stride} = (s_h, s_w) \quad \text{일때,} \\
$$
</br>
$$
\text{output dimension} \; (h_o, w_o) = ( \frac{h_i + 2p_h - h_k}{s_h} + 1, \frac{w_i + 2p_w - w_k}{s_w} + 1 )
$$

### 5. 3차원 데이터에서의 합성곱 연산

* 2차원 이미지가 여러 채널 있는 경우
* 입력 이미지의 채널 수 만큼 커널의 채널이 필요

<img src='../img/fig 7-8.png' width=50% height=50%/>
<img src='../img/fig 7-9.png' width=50% height=50%/>

* 블록으로 표현하면 다음과 같이 나타낼 수 있음

<img src='../img/fig 7-10.png' width=50% height=50%/>

* 여러개의 특성 맵을 얻고자 하면 여러개의 커널을 사용하면 됨(Fully connected layer와 같다)

<img src='../img/fig 7-13.png' width=70% height=70%/>

### 6. 구현

* 계산의 편리성을 위해 im2col함수를 사용 (대신 겹치는 영역때문에 메모리가 더 요구됨)

<img src='../img/fig 7-19.png' width=70% height=70%/>

In [2]:
def im2col(input_data, in_channel, out_h, out_w, filter_h, filter_w, stride=1, pad=0):
    '''
    순전파 계산시에 사용
    '''

    input_data = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    stack = [input_data[:,:,i:i + stride*out_h:stride,j:j + stride*out_w:stride] 
             for i in range(filter_h) for j in range(filter_w)]
    col = np.stack(stack).transpose((1,3,4,2,0)).reshape(-1, filter_h * filter_w * in_channel)

    return col

def col2im(col, input_shape, out_h, out_w, filter_h, filter_w, stride=1, pad=0):
    '''
    역전파 계산시에 사용
    '''

    n, in_channel, in_h, in_w = input_shape

    col = col.reshape(n, out_h, out_w, in_channel, filter_h, filter_w).transpose(0, 3, 1, 2, 4, 5)
    img = np.zeros((n, in_channel, in_h + 2*pad, in_w + 2*pad))
    
    for i in range(filter_h):
        for j in range(filter_w):
            img[..., i:i+stride*out_h:stride, j:j+stride*out_w:stride] += col[..., i, j]

    return img[..., pad:in_h + pad, pad:in_w + pad]

In [3]:
class Convolution:

    def __init__(self, in_channel, out_channel, kernel_size, stride=1, padding=0, W=None, b=None):

        self.in_channel = in_channel
        self.out_channel = out_channel

        self.kernel_h, self.kernel_w = kernel_size
        self.stride = stride
        self.padding = padding

        self.W = W
        self.b = b
        
        self.input_shape = None
        self.col = None
        self.dW = None
        self.db = None

    def forward(self, x):
        
        n, _, in_h, in_w = x.shape

        out_h = 1 + (in_h + 2*self.padding - self.kernel_h) // self.stride
        out_w = 1 + (in_w + 2*self.padding - self.kernel_w) // self.stride

        col = im2col(x, self.in_channel, out_h, out_w, self.kernel_h, self.kernel_w, self.stride, self.padding)
        out = np.dot(col, self.W) + self.b
        out = out.reshape(-1, out_h, out_w, self.out_channel).transpose(0, 3, 1, 2)

        self.input_shape = x.shape
        self.col = col

        return out

    def backward(self, dout):

        n, _, out_h, out_w = dout.shape
        
        dout = dout.transpose(0,2,3,1).reshape(-1, self.out_channel)
        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        dcol = np.dot(dout, self.W.T)
        dx = col2im(dcol, self.input_shape, out_h, out_w, self.kernel_h, self.kernel_w, self.stride, self.padding)

        return dx

### Ⅱ. Pooling Layer

* 영역을 집약하여 공간을 줄이는 연산
  * max pooling : 최댓값을 집약 <= 일반적으로 쓰는 pooling
  * mean pooling : 평균값으로 집약
* 학습해야할 매개변수가 없음

<img src='../img/fig 7-14.png' width=50% height=50%/>

* 입력의 변화에 영향을 적게 받음
<img src='../img/fig 7-16.png' width=50% height=50%/>

In [4]:
class Pooling:

    def __init__(self, pool_h, pool_w, stride=1, padding=0):

        self.pool_h, self.pool_w = pool_h, pool_w
        self.stride, self.padding = stride, padding
        
        self.max_idx = None

    def forward(self, x):
        
        self.input_shape = x.shape
        n, in_channel, in_h, in_w = x.shape
        
        out_h = 1 + (in_h + 2*self.padding - self.pool_h) // self.stride
        out_w = 1 + (in_w + 2*self.padding - self.pool_w) // self.stride

        col = im2col(x, in_channel, out_h, out_w, self.pool_h, self.pool_w, self.stride, self.padding)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        self.max_idx = np.argmax(col, axis=1)
        out = np.max(col, axis=1)
        out = out.reshape(n, out_h, out_w, in_channel).transpose(0,3,1,2)

        return out

    def backward(self, dout):
        
        n, in_channel, out_h, out_w = dout.shape
        dcol = dout.transpose(0,2,3,1).reshape(-1,1)

        eye = np.eye(self.pool_h * self.pool_w)
        dcol = dcol * eye[self.max_idx]
        dcol = dcol.reshape(-1, self.pool_h*self.pool_w*in_channel)

        dx = col2im(dcol, self.input_shape, out_h, out_w, self.pool_h, self.pool_w, self.stride, self.padding)

        return dx